In [1]:
import serial
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import elliptec


In [2]:
# ------------------------------
# Gaussian + derivative fitting
# ------------------------------
def gaussian(x, A, x0, sigma, offset):
    return A * np.exp(-2 * ((x - x0)**2) / (sigma**2)) + offset

def derivative_gaussian(x, A, x0, sigma):
    return -(4 * A / sigma**2) * (x - x0) * np.exp(-2 * ((x - x0)**2) / (sigma**2))


In [3]:
# ------------------------------
# Arduino voltage reader
# ------------------------------
def read_arduino_voltage(arduino, n_samples=5):
    values = []
    for _ in range(n_samples):
        try:
            line = arduino.readline().decode().strip()
            if line:
                values.append(float(line))
        except:
            continue
    if values:
        return np.mean(values)
    return None


In [4]:
# ------------------------------
# Create range between two numbers with given steps
# ------------------------------
def create_range(start, stop, step):
    """
    Create a range of values from start to stop (inclusive) with step size.
    """
    
    return np.linspace(start, stop, int((stop-start)/step))


In [5]:
# ------------------------------
#Ell20/M Thorlabs Linear stage control + Analysis
# ------------------------------
def run_scan(stage_port="COM7", arduino_port="COM10"):
    # ---- Connect to Arduino ----
    print(f"Connecting to Arduino on {arduino_port}...")
    arduino = serial.Serial(arduino_port, 115200, timeout=1)
    time.sleep(2)  # allow Arduino reset

    # ---- Connect to Elliptec Stage ----
    print(f"Connecting to Elliptec stage on {stage_port}...")
    ctrl = elliptec.Controller(stage_port)
    stage = elliptec.Linear(ctrl)

    # Home stage
    print("Homing stage...")
    stage.home()
    time.sleep(1)

    # ---- Experiment Setup ----
    start_mm = 20.0  # start position in mm
    stop_mm  = 60.0  # stop position in mm
    step_mm  = 0.1   # step size in mm
    tia_gain = 1000  # V/A

    # Generate positions
    positions_mm = create_range(start_mm, stop_mm, step_mm)
    #positions_um = [int(p*1e3) for p in positions_mm]  # µm for stage control

    powers = []

    # ---- Step Scan ----
    for pos_um, pos_mm in zip(positions_mm, positions_mm):
        stage.set_distance(pos_um)
        time.sleep(0.2)

        voltage = read_arduino_voltage(arduino, n_samples=5)
        if voltage is None:
            continue

        photocurrent = voltage / tia_gain          # I = V/Rf
        laser_power = (photocurrent / 3.8e-3) * 10 # scale to 0–10 mW

        powers.append(laser_power)

        print(f"Pos={pos_mm:.3f} mm, V={voltage:.3f} V, P={laser_power:.3f} mW")

    return positions_mm, powers, arduino, ctrl


In [6]:
# ------------------------------
# Plot Knife Edge Power
# ------------------------------
def plot_power(positions, powers):
    plt.figure()
    plt.plot(positions, powers, 'bo-', label="Power vs Position")
    plt.xlabel("Knife Edge Position (mm)")
    plt.ylabel("Laser Power (mW)")
    plt.title("Knife Edge Scan")
    plt.grid(True)
    plt.legend()
    plt.show()


In [7]:
# ------------------------------
# Derivative + Gaussian Fit
# ------------------------------
def fit_derivative(positions, powers):
    positions = np.array(positions)
    powers = np.array(powers)

    # Numerical derivative
    deriv = np.gradient(powers, positions)

    # Gaussian fit
    popt, _ = curve_fit(gaussian, positions, powers,
                        p0=[max(powers), np.mean(positions), 2.0, min(powers)])
    fit_gaussian_vals = gaussian(positions, *popt)
    fit_deriv_vals = derivative_gaussian(positions, popt[0], popt[1], popt[2])

    # Beam waist (1/e^2 radius)
    w0 = popt[2] / np.sqrt(2)

    # Plot derivative
    plt.figure()
    plt.plot(positions, deriv, 'ro-', label="dP/dx (measured)")
    plt.plot(positions, fit_deriv_vals, 'b-', label="Gaussian fit derivative")
    plt.xlabel("Knife Edge Position (mm)")
    plt.ylabel("dP/dx (mW/mm)")
    plt.title("Beam Profile Derivative")
    plt.grid(True)
    plt.legend()
    plt.show()

    print(f"Estimated beam waist (1/e^2 radius): {w0*1e3:.2f} µm")


In [ ]:
# ------------------------------
# Run the Knife Edge Scan
# ------------------------------
positions, powers, arduino, ctrl = run_scan(stage_port="COM7", arduino_port="COM10")
plot_power(positions, powers)
fit_derivative(positions, powers)

# Close connections
arduino.close()
ctrl.close()


Connecting to Arduino on COM10...
Connecting to Elliptec stage on COM7...
Controller on port COM7: Connection established!
TX: b'0in'
RX: b'0IN141200011620221401003C00000400\r\n'
Status is a dictionary.
Homing stage...
TX: b'0ho0'
RX: b'0POFFFFFFFB\r\n'
Move Successful.
TX: b'0ma00005000'
RX: b'0PO00004FFE\r\n'
Move Successful.
Pos=20.000 mm, V=4.883 V, P=12.849 mW
TX: b'0ma00005066'
RX: b'0PO00005061\r\n'
Move Successful.
Pos=20.100 mm, V=4.884 V, P=12.852 mW
TX: b'0ma000050CD'
RX: b'0PO000050CA\r\n'
Move Successful.
Pos=20.201 mm, V=4.883 V, P=12.849 mW
TX: b'0ma00005133'
RX: b'0PO0000512F\r\n'
Move Successful.
Pos=20.301 mm, V=4.883 V, P=12.849 mW
TX: b'0ma0000519A'
RX: b'0PO00005195\r\n'
Move Successful.
Pos=20.401 mm, V=4.884 V, P=12.852 mW
TX: b'0ma00005201'
RX: b'0PO000051FD\r\n'
Move Successful.
Pos=20.501 mm, V=4.883 V, P=12.849 mW
TX: b'0ma00005267'
RX: b'0PO00005262\r\n'
Move Successful.
Pos=20.602 mm, V=4.883 V, P=12.849 mW
TX: b'0ma000052CE'
RX: b'0PO000052CB\r\n'
Move Suc